# Quantized Training of LightGBM

## 現代的なGBDTの学習の流れ

（[Shi et al. (2022). Quantized training of gradient boosting decision trees.](https://arxiv.org/pdf/2207.09682.pdf)より）

勾配ブースティング決定木（GBDT）は複数の決定木を組み合わせるアンサンブル学習のアプローチをとる。

各iterationでは現状の予測値に基づくGradientとHessianを計算し、負の勾配を近似するように決定木を学習する。

$k+1$回目の反復において、現状のサンプル$i$の予測値を$\hat{y}_i^k$とすると、誤差関数$l$のgradient$g_i$とhessian$h_i$は

$$
g_i=\frac{\partial l\left(\hat{y}_i^k, y_i\right)}{\partial \hat{y}_i^k},
\quad
h_i=\frac{\partial^2 l\left(\hat{y}_i^k, y_i\right)}{\left(\partial \hat{y}_i^k\right)^2}
$$

となる。

葉$s$について、葉に含まれるデータの番号（index）の集合を$I_s$とする。葉$s$における$g_i$と$h_i$のサンプルについての合計を

$$
G_s = \sum_{i\in I_s} g_i,
\quad
H_s = \sum_{i\in I_s} h_i
$$

と表記することにすると、反復$k+1$回目において木構造が固定された下で、訓練誤差は二次のテイラー近似により

$$
\mathcal{L}_{k+1} \approx \mathcal{C}+\sum_s\left(\frac{1}{2} H_s w_s^2+G_s w_s\right)
$$

と表すことができる。

ここで$\mathcal{C}$は定数で、$w_s$は葉$s$の予測値である。近似誤差の最小化により最適値が得られる

$$
w_s^*=-\frac{G_s}{H_s}, \quad \mathcal{L}_s^*=-\frac{1}{2} \cdot \frac{G_s^2}{H_s}
$$

最適な木構造を探すのは困難であるため、木は貪欲かつ反復的に訓練される。

葉$s$を2つの子$s_1, s_2$に分割するとき、近似損失の減少分は次のように計算できる。

$$
\Delta \mathcal{L}_{s \rightarrow s_1, s_2}=\mathcal{L}_s^*-\mathcal{L}_{s_1}^*-\mathcal{L}_{s_2}^*=\frac{G_{s_1}^2}{2 H_{s_1}}+\frac{G_{s_2}^2}{2 H_{s_2}}-\frac{G_s^2}{2 H_s}
$$

葉$s$にとっての最適な分割条件の探索は、すべての特徴のすべての分割候補点を数え上げて、最も損失の減少が多いものが選ばれる。

最適分割点の探索を高速化するために用いられるのはヒストグラムである。histogram based GBDTの基本的なアイデアは特徴量の値をbinsに分割する。histogramのbinsは、そのbinに含まれるデータのgradientsとhessiansの総和が記録されている。binsの境界値のみが分割候補点になる。

> Algorithm 1 Histogram Construction for Leaf $s$
> 
> Input: Gradients $\left\{g_1, \ldots, g_N\right\}$, Hessians $\left\{h_1, \ldots, h_N\right\}$ <br>
> Input: Bin data data $[N][J]$, Data indices in leaf $s$ denoted by $I_s$ <br>
> Output: Histogram ${hist}_s$ <br>
> for $i \in I_s, j \in\{1 \ldots J\}$ do <br>
>   bin $\leftarrow \operatorname{data}[i][j]$ <br>
>   $hist_s[j][bin] . g \leftarrow$  $hist_s[j][$ bin $] . g+g_i$ <br>
>   $hist_s[j][b i n] . h \leftarrow$  $hist_s[j][b i n] . h+h_i$ <br>
> end for

伝統的には$g_i$と$h_i$には32-bit floating point numbersが使われ、histogramへの累計には32-bitか64-bitのFPが必要になる。

次節で我々は$g_i, h_i$をどのように量子化して計算コストを抑えるのかを示す。